In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from tensorflow.keras import utils

data_set = tf.keras.utils.get_file(
    fname = 'imdb.tar.gz', # download file name
    origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

def directory_data(directory):
    data={}
    data['review'] = []
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), 'r', encoding='utf-8') as file:
            data['review'].append(file.read())
    #print(data['review'][1:2])
    return pd.DataFrame.from_dict(data)

def data(directory):
    pos_df = directory_data(os.path.join(directory, 'pos'))
    neg_df = directory_data(os.path.join(directory, 'neg'))
    pos_df['sentiment'] = 1
    neg_df['sentiment'] = 0
    return pd.concat([pos_df, neg_df])

train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", 'train'))
test_df = data(os.path.join(os.path.dirname(data_set), 'aclImdb', 'test'))
imdb_pd = pd.concat([train_df, test_df])

84125825/84125825 [==============================] - 8s 0us/step


In [3]:
imdb_pd.head()

,review,sentiment
0,Tony Hawk Underground came at a point where th...,1
1,Just finished watching the movie and wanted to...,1
2,"Ralph and Mumford, misfits in their own land, ...",1
3,It's been a while since I've watched this movi...,1
4,College girl Joanne Murray takes on the unenvi...,1


In [5]:
from nltk.corpus import stopwords # 불용어 사전을 가져옴
import nltk
nltk.download('stopwords') # 불용어를 다운받음
stop_words = set(stopwords.words('english')) # 영어로된 불용어를 가져다 집합으로 만듦
# 아래의 함수는 리뷰글을 받아다가 쓸데없는 군더더기를 없애는 코드이다.
# 불용어를
def preprocessing(review, remove_stopwords=True):
    review_text = BeautifulSoup(review, 'html5lib').get_text()

    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    if remove_stopwords:
        words = review_text.split()
        words = [w for w in words if not w in stop_words]
        review_text = ' '.join(words)
    return review_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# imdb_pd에서는 review와 sentiment가 존재하는데 그 중에서 review를 위의 preprocessing 함수로 정재
# 그 목적으로 review에 해당하는 것만 imdb_pd에서 받아와서 이를 list로 만들어 list_review로 명명함
list_reviews = list(imdb_pd['review'])
# 아래의 for문에 의하여 모든 리뷰글들을 돌아다니며 preprocessing함수를 적용함
# 그 결과를 저장하는 list가 list_clean_reviews임
list_clean_reviews=[]
for review in list_reviews:
    list_clean_reviews.append(preprocessing(review))


<ipython-input-5-34217bf24249>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, 'html5lib').get_text()


In [7]:
# list_clean_reviews와 imdb_pd의 sentiment를 다시 DataFrame으로 만
list_clean_reviews_df = pd.DataFrame({'review':list_clean_reviews, 'sentiment':imdb_pd['sentiment']})
# list_clean_reviews_df에서 리뷰글과 감정평가지수를 각각 list로 만든다
list_reviews = list(list_clean_reviews_df['review'])
list_sentiments = list(list_clean_reviews_df['sentiment'])

In [8]:
# review 문자열을 수치벡터로 바꾸기 위해 TfidfVectorizer를 사용함
encoder = TfidfVectorizer(max_features=5000)
# 이진분류기 입력에 해당하는 수치벡터를 list_reviews에 기반하여 만듦(변수명 : X)
X = encoder.fit_transform(list_reviews).toarray()
# 이진분류기 입력에 해당하는 수치벡터를 list_sentiments에 기반하여 만듦(변수명 : y)
y = np.array(list_sentiments)
# 아래는 이러한 데이터가 train과 test를 모두 다 합해서 imdb_pd를 만들었기 때문에
# 다시 train과 test로 나눠야하며 그를 위하여 sklearn에 있는 train_test_split이라는 함수 사용
# 아래에 train_test_split함수에 X,y를 입력하고
# 다음은 0.2로 세팅함으로써 20% test, 80% train으로 활용
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# keras 이진 분류기
model = Sequential()
model.add(Dense(1, activation='sigmoid')) # output layer
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')
model.fit(x_train, y_train, epochs=50, verbose=1)
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: ', accuracy)

Epoch 1/50
1250/1250 [==============================] - 8s 3ms/step - loss: 0.6909 - accuracy: 0.5830
Epoch 2/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6858 - accuracy: 0.7092
Epoch 3/50
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6809 - accuracy: 0.7684
Epoch 4/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6760 - accuracy: 0.7857
Epoch 5/50
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6713 - accuracy: 0.8056
Epoch 6/50
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6667 - accuracy: 0.8156
Epoch 7/50
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6622 - accuracy: 0.8202
Epoch 8/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6577 - accuracy: 0.8215
Epoch 9/50
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6534 - accuracy: 0.8245
Epoch 10/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.649